In [413]:
import numpy as np

In [414]:
class Layer():
  def forward():
    pass

  def back_propogate():
    pass

In [415]:
class DenseLayer(Layer):
  def __init__(self,input_size,output_size):
    self.weight = np.random.rand(input_size,output_size)
    self.bias = np.random.rand(1,output_size)
    # print(self.weight)
    # print(self.bias)
  def forward(self,input):
    self.input = input
    return input @ self.weight + self.bias

  def back_propogate(self,out_grad,learning_rate=0.1):
    bias_grad = np.sum(out_grad,axis=0) / out_grad.shape[0]
    weight_grad = self.input.T @ out_grad / out_grad.shape[0]
    input_grad = out_grad @ self.weight.T
    # print(f"out_grad shape: {out_grad.shape}")
    # print(f"input shape:{self.input.shape}")
    # print(f"weight_grad shape: {weight_grad.shape}")
    self.weight-=learning_rate * weight_grad
    self.bias-=learning_rate * bias_grad
    return input_grad

In [416]:
class ReLU(Layer):
  def forward(self,input):
    self.input = input
    return np.where(input < 0 ,0,input)

  def back_propogate(self,out_grad,learning_rate=0.1):
    o= out_grad * np.where(self.input<=0,0,1)
    # print(f"in grad at relu = {o}")
    return o

In [417]:
def mse(y,y_true):
  return (y-y_true)**2

In [418]:
def mse_prime(y,y_true):
  return 2 * (y-y_true)

In [ ]:
def cross_entropy_loss(y,y_true):
  pass

In [419]:
N = 1000
feature_dimension = 20
x = np.random.rand(N,feature_dimension)
# print("X: ",x)
y_true = np.sum(np.sin(x),axis=1,keepdims=True)
# print("Y_TRUE: ",y_true)

In [420]:
network:list[Layer]= [
    DenseLayer(feature_dimension,5),
    ReLU(),
    DenseLayer(5,1)
]

In [422]:
def train(x,y_true,loss_func,loss_prime,network:list[Layer]):

  for epoch in range(1000):
    output = x
    for layer in network:
      output = layer.forward(output)
    loss = np.sum(loss_func(output,y_true),axis=0) / y_true.shape[0]
    print(f"Epoch {epoch} Loss: {loss}")
    out_grad = loss_prime(output,y_true)
    l = len(network)
    for i in range(l-1,-1,-1):
      out_grad = network[i].back_propogate(out_grad,0.001)



In [423]:
train(x,y_true,mse,mse_prime,network)

Epoch 0 Loss: [18.88199529]
Epoch 1 Loss: [6.52852492]
Epoch 2 Loss: [2.33069045]
Epoch 3 Loss: [0.87512273]
Epoch 4 Loss: [0.36514966]
Epoch 5 Loss: [0.1854512]
Epoch 6 Loss: [0.12191051]
Epoch 7 Loss: [0.09938088]
Epoch 8 Loss: [0.0913629]
Epoch 9 Loss: [0.08848649]
Epoch 10 Loss: [0.08743317]
Epoch 11 Loss: [0.0870266]
Epoch 12 Loss: [0.08684953]
Epoch 13 Loss: [0.08675396]
Epoch 14 Loss: [0.08668733]
Epoch 15 Loss: [0.08663102]
Epoch 16 Loss: [0.08657839]
Epoch 17 Loss: [0.08652711]
Epoch 18 Loss: [0.08647634]
Epoch 19 Loss: [0.08642578]
Epoch 20 Loss: [0.08637533]
Epoch 21 Loss: [0.08632494]
Epoch 22 Loss: [0.08627462]
Epoch 23 Loss: [0.08622435]
Epoch 24 Loss: [0.08617412]
Epoch 25 Loss: [0.08612395]
Epoch 26 Loss: [0.08607383]
Epoch 27 Loss: [0.08602375]
Epoch 28 Loss: [0.08597373]
Epoch 29 Loss: [0.08592376]
Epoch 30 Loss: [0.08587383]
Epoch 31 Loss: [0.08582395]
Epoch 32 Loss: [0.08577413]
Epoch 33 Loss: [0.08572435]
Epoch 34 Loss: [0.08567462]
Epoch 35 Loss: [0.08562494]
Epoc